# GenParse: Lark Interface

In [1]:
import lark

from genparse.util import LarkStuff
from arsenal import Integerizer
from arsenal.maths import compare
from collections import Counter

## Using lark as a front end

In [2]:
grammar2 = r"""
WS: /[ \t\f\r\n]/
STAR: "*"
NUMBER: /\d+/

start: WS "SELECT" WS select_expr WS "FROM" WS from_expr
  [WS "WHERE" WS bool_condition] [WS "GROUP BY" WS var_list] [WS "ORDER BY" WS orderby_expr] WS EOS
EOS: "</s>"
select_expr: STAR | select_list
bool_condition: bool_expr | "(" bool_condition WS "AND" WS bool_condition ")" | "(" bool_condition WS "OR" WS bool_condition ")" 
bool_expr: var "=" value | var ">" value | var "<" value
from_expr: "data"
orderby_expr: var_list WS "ASC" | var_list WS "DESC"
select_list: select_var ("," WS select_var)*
var_list: var ("," WS var)*
select_var: var | "AVG(" var ")" | "MEDIAN(" var ")" | "COUNT(" var ")"
var: "age" | "gender" | "year" | "state_color" | "zipcode" | "vote" | "race_ethnicity"
value: NUMBER | "red" | "blue" | "white" | "black" | "latino" | "republican" | "democrat" | "male" | "female"

"""

In [3]:
grammar1 = """
start: query_expr EOS

EOS: "</s>"

query_expr: select [ "ORDER" "BY" (order_by_expr ",")*  order_by_expr] [ "LIMIT" integer_ ] 

select: "SELECT" [(select_expr ",")*] select_expr "FROM" "data" [ "WHERE" bool_expression ] [ "GROUP" "BY" [(expression ",")*] expression ]

select_expr.0: expression_math [ [ "AS" ] alias ] -> select_expression

?expression_math: expression_product
               | expression_math PLUS expression_product -> expression_add
               | expression_math "-" expression_product -> expression_sub
               | AGGREGATION expression_math /\)/ -> sql_aggregation

?expression: (name | STAR) -> column_name
            | literal

?expression_product: expression_parens
                  | expression_product STAR expression_parens
                  | expression_product "/" expression_parens 

?expression_parens: expression
                  | /\(/ expression_parens STAR expression /\)/ 
                  | /\(/  expression_parens "/" expression /\)/ 
                  | /\(/  expression_parens PLUS expression /\)/
                  | /\(/  expression_parens "-" expression /\)/

bool_expression: bool_parentheses
                 | bool_expression "AND" bool_parentheses 
                 | bool_expression "OR" bool_parentheses
bool_parentheses: comparison_type
                 | /\(/   bool_expression "AND" comparison_type /\)/
                 | /\(/  bool_expression "OR" comparison_type /\)/
comparison_type: equals | not_equals | greater_than | less_than | greater_than_or_equal
| less_than_or_equal | is_null | is_not_null
equals: expression_math "=" expression_math
not_equals: expression_math ("<>" | "!=") expression_math
greater_than: expression_math ">" expression_math
less_than: expression_math "<" expression_math
greater_than_or_equal: expression_math ">=" expression_math
less_than_or_equal: expression_math "<=" expression_math
is_null: expression_math "is" "null"
is_not_null: expression_math "is" "not" "null"

alias: /[A-Za-z]+/
name: /[A-Za-z]+/
PLUS: /\+/

order_by_expr: expression_math ["ASC"] -> order_asc
        | expression_math "DESC" -> order_desc

AGGREGATION.8: ("sum(" | "avg(" | "min(" | "max(" | "count(" "distinct" | "count(")
STAR: /\*/
integer_: /[1-9][0-9]*/
?literal: boolean -> bool
       | integer_ -> number
       | ESCAPED_STRING -> string

boolean: "true" -> true
       | "false" -> false

%import common.WS
%ignore WS
%import common.ESCAPED_STRING
    
"""

The following code is adapted from partenon.

In [4]:
raw_grammar = grammar1

lark_stuff = LarkStuff(raw_grammar)

In [5]:
intern = Integerizer()   # rename nonterminals to integers
g = lark_stuff.convert()
g = g.rename(intern)
assert g.in_cnf()    # lark returns a grammar in CNF
#g = g.cnf

In [6]:
g

1.0: 1 → 2 3
1.0: 4 → 5 6
1.0: 7 → 8 9
1.0: 10 → 11 12
0.058823529411764705: 13 → 14 15
0.08333333333333333: 16 → 16 17
1.0: 18 → 5 19
1.0: 20 → MINUS
0.5: 21 → 21 22
0.058823529411764705: 13 → 14 23
1.0: 24 → 8 25
0.06666666666666667: 26 → 26 27
0.08333333333333333: 16 → STAR
0.1111111111111111: 28 → 26 29
1.0: 30 → 31 26
0.1: 32 → 33 34
0.058823529411764705: 35 → 36 37
1.0: 38 → 39 40
0.058823529411764705: 13 → 41 42
0.06666666666666667: 26 → FALSE
0.1: 32 → 33 43
0.058823529411764705: 35 → 33 34
0.07692307692307693: 44 → 26 45
1.0: 46 → 47 48
0.058823529411764705: 35 → 33 43
1.0: 49 → 32 50
1.0: 51 → 21 52
1.0: 53 → STAR
0.058823529411764705: 5 → __ANON_7
1.0: 54 → 55 56
1.0: 57 → 58 40
1.0: 59 → 5 60
1.0: 61 → 62 63
0.09090909090909091: 3 → 26 10
0.1111111111111111: 28 → 26 64
0.08333333333333333: 14 → 41 42
0.058823529411764705: 5 → 36 37
0.08333333333333333: 16 → 16 65
0.5: 66 → 35 67
0.1: 32 → __ANON_6
1.0: 68 → EQUAL
0.058823529411764705: 5 → 33 34
1.0: 33 → __ANON_1
0.1111111111111111: 28 → 26 30
0.07692307692307693: 44 → 26 69
0.058823529411764705: 35 → __ANON_6
0.058823529411764705: 35 → 26 70
0.25: 71 → __ANON_7
0.16666666666666666: 40 → __ANON_6
0.06666666666666667: 26 → ESCAPED_STRING
1.0: 72 → 5 73
0.058823529411764705: 13 → 14 74
0.058823529411764705: 13 → 41 72
0.058823529411764705: 5 → 33 43
0.1111111111111111: 28 → 26 75
1.0: 76 → 77 78
0.08333333333333333: 14 → 41 79
1.0: 6 → 77 80
1.0: 81 → 21 82
1.0: 83 → 8 84
1.0: 85 → 8 61
1.0: 29 → 86 26
1.0: 87 → 88 89
0.058823529411764705: 13 → 41 81
0.08333333333333333: 14 → 41 90
0.1111111111111111: 28 → 26 91
0.058823529411764705: 5 → __ANON_6
0.058823529411764705: 5 → 26 70
0.1: 32 → FALSE
1.0: 92 → 32 93
1.0: 37 → 26 94
0.058823529411764705: 35 → 26 95
1.0: 88 → OR
1.0: 69 → 96 26
0.058823529411764705: 35 → FALSE
0.16666666666666666: 40 → FALSE
0.09090909090909091: 3 → 26 75
0.5: 21 → 5 67
0.08333333333333333: 16 → __ANON_7
1.0: 41 → SELECT
1.0: 34 → 32 97
0.058823529411764705: 5 → 33 49
1.0: 98 → 58 40
1.0: 99 → 8 100
0.058823529411764705: 13 → 41 59
0.06666666666666667: 26 → STAR
1.0: 101 → 26 102
0.09090909090909091: 3 → 26 102
0.058823529411764705: 5 → 26 27
0.08333333333333333: 16 → 33 34
1.0: 103 → 104 105
0.058823529411764705: 13 → 41 106
1.0: 9 → 62 107
0.058823529411764705: 13 → 41 108
1.0: 73 → 77 24
1.0: 11 → IS
0.1: 32 → ESCAPED_STRING
0.08333333333333333: 16 → 33 43
0.058823529411764705: 5 → FALSE
1.0: 100 → 55 109
0.058823529411764705: 5 → TRUE
0.058823529411764705: 35 → ESCAPED_STRING
0.1111111111111111: 28 → 26 10
0.16666666666666666: 40 → ESCAPED_STRING
1.0: 110 → 77 8
1.0: 111 → __ANON_7
1.0: 112 → 5 76
1.0: 113 → LESSTHAN
1.0: 15 → 114 115
1.0: 116 → 77 117
1.0: 80 → 8 118
0.09090909090909091: 3 → 33 119
1.0: 0 → 13 120
0.058823529411764705: 13 → 14 121
1.0: 47 → AS
1.0: 74 → 114 122
0.08333333333333333: 14 → 41 106
1.0: 42 → 5 123
0.5: 58 → 58 124
0.058823529411764705: 5 → 26 46
1.0: 125 → 39 126
0.25: 71 → TRUE
1.0: 127 → 66 128
0.058823529411764705: 13 → 41 129
1.0: 130 → 26 64
1.0: 131 → 66 35
0.058823529411764705: 35 → 26 132
1.0: 133 → 26 69
1.0: 94 → __ANON_0
0.058823529411764705: 13 → 41 18
0.07692307692307693: 44 → 33 134
1.0: 135 → 77 7
1.0: 136 → 44 54
1.0: 27 → 137 16
0.07692307692307693: 44 → 26 29
1.0: 138 → 8 139
1.0: 31 → __ANON_5
1.0: 104 → SLASH
1.0: 140 → 77 83
1.0: 122 → 39 141
0.08333333333333333: 16 → 33 49
1.0: 142 → 35 67
0.058823529411764705: 35 → 16 17
1.0: 77 → FROM
1.0: 8 → DATA
1.0: 19 → 77 99
0.09090909090909091: 3 → 26 45
1.0: 97 → 53 143
0.1: 32 → STAR
1.0: 79 → 21 144
1.0: 108 → 5 110
0.058823529411764705: 35 → STAR
1.0: 65 → 53 32
1.0: 145 → 55 146
0.08333333333333333: 16 → FALSE
0.08333333333333333: 16 → TRUE
1.0: 106 → 21 4
0.07692307692307693: 44 → 26 64
1.0: 147 → 39 127
0.06666666666666667: 26 → __ANON_7
1.0: 43 → 32 103
0.06666666666666667: 26 → 33 92
1.0: 48 → __ANON_6
0.1111111111111111: 28 → 26 102
1.0: 148 → 62 44
1.0: 134 → 44 149
0.5: 150 → TRUE
0.07692307692307693: 44 → 26 30
0.08333333333333333: 

In [7]:
len(g.rules), len(g.V), len(g.N)

(320, 38, 184)

In [8]:
sorted(g.cnf.V)

['AGGREGATION',
 'AND',
 'AS',
 'ASC',
 'BY',
 'COMMA',
 'DATA',
 'DESC',
 'EOS',
 'EQUAL',
 'ESCAPED_STRING',
 'FALSE',
 'FROM',
 'GROUP',
 'IS',
 'LESSTHAN',
 'LIMIT',
 'MINUS',
 'MORETHAN',
 'NOT',
 'NULL',
 'OR',
 'ORDER',
 'PLUS',
 'SELECT',
 'SLASH',
 'STAR',
 'TRUE',
 'WHERE',
 'WS',
 '__ANON_0',
 '__ANON_1',
 '__ANON_2',
 '__ANON_3',
 '__ANON_4',
 '__ANON_5',
 '__ANON_6',
 '__ANON_7']

In [9]:
#from newton.linking import LinkAnalysis
#f = Integerizer()
#links = LinkAnalysis(g.rename(f))
#links.dfs

In [10]:
#g.language(6)

## Tokenization

We can extract lark's tokenizer in a format that we can build on.  We will even make a DIY tokenizer based on Python's `re` library.

| Terminology  |         |
|--------------|---------|
| tokenization | lexing  |
| tokenizers   | lexers  |
| tokens       | lexemes |



In [11]:
sorted(lark_stuff.all_terminals, key=lambda t: -t.priority)

[TerminalDef('AGGREGATION', '(?:count\\(distinct|count\\(|sum\\(|avg\\(|min\\(|max\\()'),
 TerminalDef('ESCAPED_STRING', '".*?(?<!\\\\)(\\\\\\\\)*?"'),
 TerminalDef('WS', '(?:[ \t\x0c\r\n])+'),
 TerminalDef('EOS', '</s>'),
 TerminalDef('PLUS', '\\+'),
 TerminalDef('STAR', '\\*'),
 TerminalDef('COMMA', ','),
 TerminalDef('ORDER', 'ORDER'),
 TerminalDef('BY', 'BY'),
 TerminalDef('LIMIT', 'LIMIT'),
 TerminalDef('WHERE', 'WHERE'),
 TerminalDef('GROUP', 'GROUP'),
 TerminalDef('SELECT', 'SELECT'),
 TerminalDef('FROM', 'FROM'),
 TerminalDef('DATA', 'data'),
 TerminalDef('AS', 'AS'),
 TerminalDef('MINUS', '\\-'),
 TerminalDef('__ANON_0', '\\)'),
 TerminalDef('SLASH', '/'),
 TerminalDef('__ANON_1', '\\('),
 TerminalDef('AND', 'AND'),
 TerminalDef('OR', 'OR'),
 TerminalDef('EQUAL', '='),
 TerminalDef('__ANON_2', '<>'),
 TerminalDef('__ANON_3', '!='),
 TerminalDef('MORETHAN', '>'),
 TerminalDef('LESSTHAN', '<'),
 TerminalDef('__ANON_4', '>='),
 TerminalDef('__ANON_5', '<='),
 TerminalDef('IS', 'i

### DIY tokenizer

In [12]:
text = "12 + 24 - 36 * 48 / 60 SELECT table.name AS thing WHERE table.potato IS NOT 'banana'"

for x, y in lark_stuff.simple_tokenizer(text):
    print(f'{x:15s} -> {y!r}')

__ANON_7        -> '12'
PLUS            -> '+'
__ANON_7        -> '24'
MINUS           -> '-'
__ANON_7        -> '36'
STAR            -> '*'
__ANON_7        -> '48'
SLASH           -> '/'
__ANON_7        -> '60'
SELECT          -> 'SELECT'
__ANON_6        -> 'table'
__ANON_6        -> 'name'
AS              -> 'AS'
__ANON_6        -> 'thing'
WHERE           -> 'WHERE'
__ANON_6        -> 'table'
__ANON_6        -> 'potato'
__ANON_6        -> 'IS'
__ANON_6        -> 'NOT'
__ANON_6        -> 'banana'


### Parsing tokenized input

In [13]:
text = 'SELECT name FROM data </s>'

In [ ]:
tokens = list(lark_stuff.lex(text))
tokens

Call the lark parser on the text:

In [ ]:
lark_stuff.instance.parse(text)

We can call the lark parser on these tokens:

In [ ]:
lark_stuff.parser.parse(tokens, 'start')

We can call our parser on this text to get its total weight

In [ ]:
g([t.type for t in tokens])

### Tokenizer State Machines

**TODO**: Dive into the [greenery](https://github.com/qntm/greenery) FSMs to figure out how they work; we can work backward from the Partenon tensor building routine.  Another option might be the [interegular](https://github.com/MegaIng/interegular) library.


**Note**: Tokenizers are FSTs, not FSAs.  However, these libraries implement the kind of restricted FSTs with a separate FSA per token type.

In [15]:
import greenery

def make_greenery_fsms(
    regex_list, 
    ignore = ("\s*",), 
    chars = tuple(chr(i) for i in range(128))
):
    match ignore:
        case []:
            ignore_regex = ""
        case [ignore]:
            ignore_regex = ignore
        case _:
            raise ValueError("ignore must be a list of length at most 1")

    patterns = []
    for regex in regex_list:
        # greenery does not escape spaces
        regex = regex.replace("\\ ", " ")
        patterns.append(greenery.parse(regex + ignore_regex))

    return [pattern.to_fsm() for pattern in patterns]

In [16]:
#fsms = make_greenery_fsms([t for t in tokens])

In [17]:
#fsms[0]

The code below was used in [partenon](https://github.com/probcomp/partenon) to convert the the lark tokenizer `->` greenery FSMs `->` tensors.  We can repurpose it to convert into a transducer from characters to token names.

In [18]:
# def make_matrix_from_fsms(fsms, chars):
#     max_states = max([len(fsm.states) for fsm in fsms])
#     n_inputs = len(chars)
#     m_shape = (len(fsms), max_states, n_inputs, max_states)
#     m = np.zeros(m_shape, dtype=np.int8)
#     finals = jnp.zeros((len(fsms), max_states))
#     for fsm_idx, fsm in enumerate(fsms):
#         for final_state in fsm.finals:
#             finals = finals.at[fsm_idx, final_state].set(1)
#         fsm_states = fsm.states
#         rejection_states = [e for e in fsm_states if not fsm.islive(e)]
#         for state in fsm_states:
#             arcs = fsm.map[state]
#             for input_char, next_state in arcs.items():
#                 if next_state in rejection_states:  # rejection state
#                     continue
#                 for char in input_char.get_chars():
#                     input_idx = chars.index(char)
#                     m[fsm_idx, state, input_idx, next_state] = 1
#     m = jnp.array(m)
#     return m, finals, max_states

In [19]:
import fsa

In [35]:
i = 4
token_class = lark_stuff.all_terminals[i]
[fsm] = make_greenery_fsms([token_class.pattern.to_regexp()])

m = fsa.FSA()
m.add_start(fsm.initial)
for final_state in fsm.finals:
    m.add_stop(final_state)
rejection_states = [e for e in fsm.states if not fsm.islive(e)]
for state in fsm.states:
    arcs = fsm.map[state]
    for input_char, next_state in arcs.items():
        if next_state in rejection_states:  # rejection state
            continue
        for char in input_char.get_chars():
            m.add(state, char, next_state)

display(token_class)
display(m)

TerminalDef('AGGREGATION', '(?:count\\(distinct|count\\(|sum\\(|avg\\(|min\\(|max\\()')

(a⋅v⋅g + m⋅a⋅x + m⋅i⋅n + s⋅u⋅m)⋅( + (a⋅v⋅g + m⋅a⋅x + m⋅i⋅n + s⋅u⋅m)⋅(⋅(	 + 
 + 
 + 
 + 
 +  )*⋅(	 + 
 + 
 + 
 + 
 +  ) + c⋅o⋅u⋅n⋅t⋅(⋅(	 + 
 + 
 + 
 + 
 +   + (	 + 
 + 
 + 
 + 
 +  )⋅(	 + 
 + 
 + 
 + 
 +  )*⋅(	 + 
 + 
 + 
 + 
 +  )) + c⋅o⋅u⋅n⋅t⋅(⋅d⋅i⋅s⋅t⋅i⋅n⋅c⋅(t + t⋅(	 + 
 + 
 + 
 + 
 +  )*⋅(	 + 
 + 
 + 
 + 
 +  )) + c⋅o⋅u⋅n⋅t⋅(

In [36]:
#for e in m.arcs(): print(e)

## Prefix Grammar

In [ ]:
len(g.cnf.rules), len(g.cnf.prefix_grammar.trim().rules), len(g.cnf.prefix_grammar.trim().rules)/len(g.cnf.rules)

In [ ]:
#N = G.nullaryremove()    # could be faster with SCC-based prioritization

In [ ]:
g.cnf.prefix_grammar.trim().cnf

In [ ]:
g.prefix_weight([t.type for t in tokens])